In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import libs
import os
import glob, pylab, pandas as pd
import pydicom, numpy as np
import random
import json
import time
import copy
import pydicom
import torchvision
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
from matplotlib import patches, patheffects

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torchvision import datasets, transforms
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
from pathlib import Path

import torch.nn.functional as F

# from fastai.conv_learner import *
# from fastai.dataset import *
# from fastai.dataset import ImageClassifierData

In [ ]:
!pip install pytorch_pretrained_vit

In [ ]:
from pytorch_pretrained_vit import ViT
model = ViT('B_16_imagenet1k', pretrained=True)

In [ ]:
import json
from PIL import Image
import torch
from torchvision import transforms

# Load ViT
from pytorch_pretrained_vit import ViT
model = ViT('B_16_imagenet1k', pretrained=True)
model.eval()

# Load image
# NOTE: Assumes an image `img.jpg` exists in the current directory
img = transforms.Compose([
    transforms.Resize((384, 384)), 
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
])(Image.open('img.jpg')).unsqueeze(0)
print(img.shape) # torch.Size([1, 3, 384, 384])

# Classify
with torch.no_grad():
    outputs = model(img)
print(outputs.shape)  # (1, 1000)

In [ ]:
!ls ../input/rsna-pneumonia-detection-challenge

In [ ]:
!ls ../input/rsna-pneumonia-detection-challenge/stage_2_train_images | head

In [ ]:
PATH = '../input/rsna-pneumonia-detection-challenge/'
df = pd.read_csv(PATH + 'stage_2_train_labels.csv')
df.head()

In [ ]:
# print(df.drop_duplicates('patientId').shape)
# len(list((PATH + 'stage_2_train_images').iterdir()))
print(len(os.listdir(PATH + 'stage_2_train_images')))
df = df.drop_duplicates('patientId').reset_index(drop=True)
print(df.shape)
print(df.head())

target label to one hot vector

In [ ]:
# encoder = LabelEncoder()
# encoder.fit(df['Target'])
# df['label'] = encoder.transform(df['Target'])
# print(df.head())

앙상블을 함으로써 performance를 높일 수 있다.
그러나 train 시간이 더 길어진다.
train data를 3개로 나누고 training후 majority vote로 하자.

In [ ]:
# df = df[0:2000]
train_df, test_df = train_test_split(df, test_size=0.1)
train2_df, train3_df = train_test_split(train_df, test_size=1/3)
train1_df, train2_df = train_test_split(train2_df, test_size=0.5)

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
train1_df.reset_index(drop=True, inplace=True)
train2_df.reset_index(drop=True, inplace=True)
train3_df.reset_index(drop=True, inplace=True)
print(train_df.shape)
print(len(train_df), len(train1_df), len(train2_df), len(train3_df))
# train_df.head()

data중에서 label 1과 0의 비율은 약 1:4 로 0이 많다.
train/test, 및 train을 각각 3개로 나눌 때도 이 비율이 유지됨을 볼 수 있다.

dataset을 만들때 비율을 고려하여 sampling을 하는 방안을 생각할 수 있다.
또는 loss를 focal loss로 쓰면서 label 0에 대한 학습시 가중치 update를 낮추는 법도 가능하다.

In [ ]:
print(len(df[df['Target']==1])/len(df))
print(len(train_df[train_df['Target']==1])/len(train_df))
print(len(test_df[test_df['Target']==1])/len(test_df))
print(len(train1_df[train1_df['Target']==1])/len(train1_df))
print(len(train2_df[train2_df['Target']==1])/len(train2_df))


In [ ]:
class MDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transforms.Compose(transform) if transform else None
        self.dir = PATH + 'stage_2_train_images/'

    def __len__(self):
        return len(self.df)
    
    def read_dicom_image(self, loc):
        # return numpy array
        img_arr = pydicom.read_file(loc).pixel_array
        img_arr = img_arr/img_arr.max()
        img_arr = (255*img_arr).clip(0, 255).astype(np.uint8)
        img_arr = Image.fromarray(img_arr).convert('RGB') # model expects 3 channel image
        return img_arr    

    def __getitem__(self, idx):
        pid = self.df.iloc[idx, 0]
#         print(pid)
        pimage = self.read_dicom_image(self.dir + pid + '.dcm')
        if self.transform:
            pimage = self.transform(pimage)
        label = self.df.iloc[idx, 5]
        return pid, pimage, label

In [ ]:
BATCH_SIZE = 32
transform = [transforms.Resize(224), transforms.RandomHorizontalFlip() , transforms.ToTensor()]

train_dataset = MDataset(train_df, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_dataset = MDataset(test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

train1_dataset = MDataset(train1_df, transform=transform)
train1_loader = DataLoader(train1_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

train2_dataset = MDataset(train2_df, transform=transform)
train2_loader = DataLoader(train2_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

train3_dataset = MDataset(train3_df, transform=transform)
train3_loader = DataLoader(train3_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# imbalanced dataset sampler
1:1 ratio per class


In [ ]:
# # # imbalanced dataset sampler
# # !ls imbalanced-dataset-sampler
# # !rm -rf imbalanced-dataset-sampler
# # !git clone https://github.com/ufoym/imbalanced-dataset-sampler
# # !pip install imbalanced-dataset-sampler/

# import torch
# import torch.utils.data
# import torchvision


class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices (list, optional): a list of indices
        num_samples (int, optional): number of samples to draw
        callback_get_label func: a callback-like function which takes two arguments - dataset and index
    """

    def __init__(self, dataset, indices=None, num_samples=None, callback_get_label=None):
                
        # if indices is not provided, 
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset))) \
            if indices is None else indices

        # define custom callback
        self.callback_get_label = callback_get_label

        # if num_samples is not provided, 
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) \
            if num_samples is None else num_samples
            
        # distribution of classes in the dataset 
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1
                
        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)]
                   for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, idx):
#         return dataset.train_labels[idx].item()
        return dataset.df.loc[idx].Target
                
    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

# from torchsampler import ImbalancedDatasetSampler

imbalanced_train_loader = DataLoader(
    train_dataset, 
    sampler=ImbalancedDatasetSampler(train_dataset),
    batch_size=32, 
    shuffle=False, 
#     num_workers=2
)

# import sys
# for _, pid, label in imbalanced_train_loader:
# #     print(pid, label)
#     print(label)
#     sys.exit()

In [ ]:
#https://www.kaggle.com/c/tgs-salt-identification-challenge/discussion/65938
# focal loss for unbalancing label
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduce=True):
        super(FocalLoss, self).__init__()
#         self.device = torch.device('cuda')
#         self.alpha = alpha.to(self.device)
        self.alpha =alpha
        self.gamma = gamma
        self.reduce = reduce
        self.loss = F.cross_entropy
#         print(alpha)
#         print(type(alpha))

    def forward(self, inputs, targets):
        CE_loss = self.loss(inputs, targets, reduce=False)
        pt = torch.exp(-CE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * CE_loss
#         CE_loss = self.loss(inputs, targets, reduce=False, weight=self.alpha)
#         pt = torch.exp(-(self.loss(inputs, targets, reduce=False)))
# #         pt = torch.exp(-CE_loss)
#         F_loss = (1-pt)**self.gamma * CE_loss        

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss    

In [ ]:
# https://gist.github.com/SuperShinyEyes/dcc68a08ff8b615442e3bc6a9b55a354
class F1_Loss(nn.Module):
    '''Calculate F1 score. Can work with gpu tensors
    
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Returns
    -------
    torch.Tensor
        `ndim` == 1. epsilon <= val <= 1
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    - http://www.ryanzhang.info/python/writing-your-own-loss-function-module-for-pytorch/
    '''
    def __init__(self, epsilon=1e-7):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true,):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, 2).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)
        
        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        return 1 - f1.mean()

In [ ]:

model= torchvision.models.resnet50(pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2) # target label is 2
criterion = nn.CrossEntropyLoss()
# criterion = FocalLoss(alpha=0.97, gamma=2, reduce=True)
# criterion = F1_Loss()
# model_ft = model.cuda()

# save for ensemble
default_model = copy.deepcopy(model.state_dict())

# Observe that all parameters are being optimized
optimizer = optim.Adam(model.parameters(), lr=0.00008)

dev = "cuda"
# dev = "cpu"
device = torch.device(dev)
model.to(device)
criterion = criterion.to(device)

# # Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

scheduler = lr_scheduler.LambdaLR(
    optimizer=optimizer, lr_lambda=lambda epoch: 1 / (epoch + 1)
)

In [ ]:

def model_eval(model, dev='cuda') :
    device = torch.device(dev)
#     test_criterion = F1_Loss()
    test_criterion = nn.CrossEntropyLoss()
    test_criterion = test_criterion.to(device)
    model.to(device)
    model.train(False)
    data_loader = test_loader
    running_corrects = 0
    test_loss = 0
    
    # accuracy and loss
    with torch.no_grad():    
        for _, pimages, labels in data_loader:
            pimages = torch.tensor(pimages)
            labels = torch.tensor(labels)
            pimages, labels = pimages.to(device), labels.to(device)  
            outputs = model(pimages)  
            loss = test_criterion(outputs, labels)
            _, preds = torch.max(outputs.data, 1)  
            running_corrects += torch.sum(preds == labels.data)
            test_loss += (loss*pimages.size()[0])
        epoch_acc = running_corrects / len(data_loader.dataset)
#     print('{} loss: {:.4f} Acc: {:.4f}'.format(
#         'test',test_loss/len(data_loader.dataset), epoch_acc))   
    
    return (epoch_acc, test_loss/len(data_loader.dataset))


기본 테스트

In [ ]:
def model_train(train_loader, model) :
    num_epochs = 10
    # model.train(True)
    best_test_acc = 0
    best_f1_loss = 100

    train_loss = []
    test_loss = []

    best_model_wts = copy.deepcopy(model.state_dict())
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 50)
        start = time.time()
        model.train(True)

        running_loss = 0.0
        running_corrects = 0
        data_loader = train_loader
        for i, data in enumerate(data_loader):
            _, pimages, labels = data
            pimages = torch.tensor(pimages)
            labels = torch.tensor(labels)
            pimages, labels = pimages.to(device), labels.to(device)

            outputs = model(pimages)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)

            # statistics
            running_loss += (loss) * pimages.size()[0]
            running_corrects += torch.sum(preds == labels.data)  
                
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()                

        epoch_loss = running_loss / len(data_loader.dataset)
        train_loss.append(float(epoch_loss))
        epoch_acc = running_corrects / len(data_loader.dataset)

        scheduler.step()

        test_acc, f1_loss = model_eval(model, device)
        test_loss.append(float(f1_loss))
        
        print('train Loss: {:.4f} test Loss: {:.4f} train Acc: {:.4f} test Acc: {:.4f}'.format(
            epoch_loss, f1_loss, epoch_acc, test_acc))  
        print('time per epoch :', time.time() - start)        
        if test_acc > best_test_acc :
            best_test_acc = test_acc
#         if f1_loss < best_f1_loss :
#             best_f1_loss = f1_loss
            # load best model weights
            best_model_wts = model.state_dict()
            print('best model is updated')

        print('-' * 50)

    model.load_state_dict(best_model_wts)
    test_acc, f1_loss = model_eval(model, device)
    print('fl_score :', 1-f1_loss)
    return model, (train_loss, test_loss)


In [ ]:
model, losses = model_train(train1_loader, model)

from matplotlib import pyplot as plt
train_lo, test_lo = losses
epochs = range(10)
# list(train_lo)
plt.plot(epochs, train_lo)
plt.plot(epochs, test_lo)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'])
plt.show()

In [ ]:
model.load_state_dict(default_model)
optimizer = optim.Adam(model.parameters(), lr=0.00008)
scheduler = lr_scheduler.LambdaLR(
    optimizer=optimizer, lr_lambda=lambda epoch: 1 / (epoch + 1)
)
model, _ = model_train(imbalanced_train_loader, model)    

In [ ]:
# save model
torch.save(model.state_dict(), 'best_model')
!tar -zcvf best_model.tar.gz best_model
!ls -al

앙상블 테스트

In [ ]:
models = []

# training 3 models
for data_loader in [train1_loader, train2_loader, train3_loader] :
    print('*' * 100)
#     print(data_loader)

    model.load_state_dict(default_model)
    optimizer = optim.Adam(model.parameters(), lr=0.00008)
    scheduler = lr_scheduler.LambdaLR(
        optimizer=optimizer, lr_lambda=lambda epoch: 1 / (epoch + 1)
    )
    
    model, _ = model_train(data_loader, model)
    models.append(copy.deepcopy(model.state_dict()))

In [ ]:
# save model
import pickle

# save
with open('models.pickle', 'wb') as f:
    pickle.dump(models, f, pickle.HIGHEST_PROTOCOL)

# # load
# with open('models.pickle', 'rb') as f:
#     models = pickle.load(f)

!ls -al

In [ ]:
# test_df.Target[0:20]

In [ ]:
# ensenble test
import math

preds = []
for m in models :
    model.load_state_dict(m)
    model.train(False)
    data_loader = test_loader
    running_corrects = 0
    pred = []

    # accuracy and loss
    with torch.no_grad():    
        for _, pimages, labels in data_loader:
            pimages = torch.tensor(pimages)
            labels = torch.tensor(labels)
            pimages, labels = pimages.to(device), labels.to(device) 
            outputs = model(pimages)
           
            _, p = torch.max(outputs.data, 1)
            pred.append(p)
        preds.append(pred)
#         print(preds)

# print(len(models))
# print(preds)
answer = []
# majority voting
for a, b, c in zip(preds[0], preds[1], preds[2]) :
    for i, j, k in zip(a, b, c) :
        answer.append(round(int(i+j+k)/3))
# print(answer)


![](http://)Recall = TP / (TP + FN)  
Precision = TP / (TP + FP)  
Accuracy = (TP + TN) / (TP + FP + FN + TN)  
F1-Score = 2 * (Recall * Precision) / (Recall + Precision)

In [ ]:
# print(answer)
# print(test_df.Target[0:20])

In [ ]:
# confusion matrix and F1 score of ensenble
TP, FP, TN, FN = 0,0,0,0
for a, p in zip(test_df.Target, answer) :
    if a == 1 and p == 1 :
        TP += 1
    elif a == 1 and p == 0 :
        FN += 1
    elif a == 0 and p == 1 :
        FP += 1
    elif a == 0 and p == 0 :
        TN += 1

print('TP {}, FP {}, TN {}, FN {}'.format(TP, FP, TN, FN))   
Recall = TP / (TP + FN)
Precision = TP / (TP + FP)
Accuracy = (TP + TN) / (TP + FP + FN + TN)
F1_Score = 2 * (Recall * Precision) / (Recall + Precision)
print('Recall {:.4} precision {:.4}, accuracy {:.4}, F1 score {:.4}'.format(
    Recall, Precision, Accuracy, F1_Score))


In [ ]:
!jupyter nbconvert --to html __notebook_source__.ipynb

In [ ]:
!ls /kaggle/input